# **Product Recognition of Food Products**

## Image Processing and Computer Vision - Assignment Module \#1


Contacts:

- Prof. Giuseppe Lisanti -> giuseppe.lisanti@unibo.it
- Prof. Samuele Salti -> samuele.salti@unibo.it
- Alex Costanzino -> alex.costanzino@unibo.it
- Francesco Ballerini -> francesco.ballerini4@unibo.it


Computer vision-based object detection techniques can be applied in super market settings to build a system that can identify products on store shelves.
An example of how this system could be used would be to assist visually impaired customers or automate common store management tasks like detecting low-stock or misplaced products, given an image of a shelf in a store.

## Task
Develop a computer vision system that, given a reference image for each product, is able to identify such product from one picture of a store shelf.

<figure>
<a href="https://imgbb.com/">
  <center>
  <img src="https://i.ibb.co/TwkMWnH/Screenshot-2024-04-04-at-14-54-51.png" alt="Screenshot-2024-04-04-at-14-54-51" border="0" width="300" />
</a>
</figure>

For each type of product displayed in the
shelf the system should report:
1. Number of instances;
1. Dimension of each instance (width and height in pixel of the bounding box that enclose them);
1. Position in the image reference system of each instance (center of the bounding box that enclose them).

#### Example of expected output
```
Product 0 - 2 instance found:
  Instance 1 {position: (256, 328), width: 57px, height: 80px}
  Instance 2 {position: (311, 328), width: 57px, height: 80px}
Product 1 – 1 instance found:
.
.
.
```

### Track A - Single Instance Detection
Develop an object detection system to identify single instance of products given one reference image for each item and a scene image.

The system should be able to correctly identify all the product in the shelves
image.

### Track B - Multiple Instances Detection
In addition to what achieved at step A, the system should also be able to detect multiple instances of the same product.

## Data
Two folders of images are provided:
* **Models**: contains one reference image for each product that the system should be able to identify.
* **Scenes**: contains different shelve pictures to test the developed algorithm in different scenarios. The images contained in this folder are corrupted by noise.

#### Track A - Single Instance Detection
* **Models**: {ref1.png to ref14.png}.
* **Scenes**: {scene1.png to scene5.png}.

#### Track B - Multiple Instances Detection
* **Models**: {ref15.png to ref27.png}.
* **Scenes**: {scene6.png to scene12.png}.

In [ ]:
from importlib.util import find_spec


try:
    from google.colab import drive
    drive.mount('/content/drive')

    !cp -r /content/drive/MyDrive/AssignmentsIPCV/dataset.zip ./
    !unzip dataset.zip
except ModuleNotFoundError:
    pass

## Evaluation criteria
1. **Procedural correctness**. There are several ways to solve the assignment. Design your own sound approach and justify every decision you make;

2. **Clarity and conciseness**. Present your work in a readable way: format your code and comment every important step;

3. **Correctness of results**. Try to solve as many instances as possible. You should be able to solve all the instances of the assignment, however, a thoroughly justified and sound procedure with a lower number of solved instances will be valued **more** than a poorly designed approach.

# ASSIGNMENT:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

models_path="dataset/models/"
scenes_path="dataset/scenes/"

products_a=[f"{models_path}ref{i}.png" for i in range(1,15)]
products_b=[f"{models_path}ref{i}.png" for i in range(15,28)]

scenes_a=[f"{scenes_path}scene{i}.png" for i in range(1,6)]
scenes_b=[f"{scenes_path}scene{i}.png" for i in range(6,13)]

In [ ]:
fig, axes = plt.subplots(2, 7, figsize=(14, 4))
axes = axes.flatten()

for i, image in enumerate(products_b):
    axes[i].imshow(cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB))
    axes[i].axis('off')
    axes[i].set_title(image[len(models_path):])
    if i==len(products_b)-1:
        for ax in axes[i+1:]:
            ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 7, figsize=(14, 4))
axes = axes.flatten()

for i, image in enumerate(scenes_b):
    axes[i].imshow(cv2.cvtColor(cv2.imread(image), cv2.COLOR_BGR2RGB))
    axes[i].axis('off')
    axes[i].set_title(image[len(scenes_path):])

plt.tight_layout()
plt.show()

## TRACK A

In [ ]:
MAX=225

def window_elements(P, W, i, j):
    elements=P[max(0,i-(W-1)//2):min(P.shape[0]-1,i+(W-1)//2)+1,max(0,j-(W-1)//2):min(P.shape[1]-1,j+(W-1)//2)+1]
    return elements.reshape((np.prod(elements.shape),))


def uncorrupted(a):
    return a[(a > 0) & (a < MAX)]


def find_median(a):
    return np.median(a)


def find_mean(a):
    return np.mean(a)


def only_0_and_MAX(a):
    for element in a:
        if element!=0 and element !=MAX:
            return False
    return True


def only_0(a):
    for element in a:
        if element!=0:
            return False
    return True


def only_MAX(a):
    for element in a:
        if element!=MAX:
            return False
    return True

def adaptive_approach(P):
    W=3
    h=2
    Wmax=9
    width,height=P.shape
    for i in range(width):
        for j in range(height):
            # print(j)
            if P[i,j]>0 and P[i,j]<MAX:
                # print('non corrotto', P[i,j])
                continue
            v=window_elements(P,W,i,j)
            V=uncorrupted(v)
            N=len(V)
            while W<=Wmax:
                if N>=W:
                    # print('n>=W', P[i,j])
                    P[i,j]=find_median(V)
                    break
                if W<Wmax:
                    if N<W: #case I
                        W+=h
                        v=window_elements(P,W,i,j)
                        V=uncorrupted(v)
                        N=len(V)
                        # print(W,N)
                        # print('1')
                    else: #case II
                        P[i,j]=find_median(V)
                        W=3
                        # print('2')
                        break
                if W==Wmax:
                    if N<W and N!=0: #caseIII
                        P[i,j]=find_mean(V)
                        # print('3')
                        break
                    if only_0_and_MAX(v): #caseIV
                        P[i,j]=find_mean(v)
                        # print('4')
                        break
                    if only_0(v): #caseV
                        P[i,j]=MAX
                        # print('5')
                        break
                    if only_MAX(v): #caseVI
                        P[i,j]=0
                        # print('6')
                        break
    return P


def adaptive_davide(P):
    h=2
    Wmax=9
    width,height=P.shape
    
    for i in range(width):
        for j in range(height):
            W=3
            if P[i][j] > 0 and P[i][j] < MAX:
                continue
            
            while W <= Wmax:
                v = window_elements(P, W, i, j)
                V = uncorrupted(v)
                N = len(V)
                # print(f'P:{i,j} N:{N}, W:{W}')
                # print(f'V:{np.sort(V)}')
                if N>=W:
                    P[i][j]=find_median(V)
                    # print('n>=W')
                    break
                else:
                    if W < Wmax:
                        if N < W: #case I
                            W += h
                            # print('Case1')
                            continue
                        else: #case II
                            P[i][j] = find_median(V)
                            W = 3
                            print('Case2')
                            break
                    
                    if W==Wmax:
                        if N < W and N != 0: #caseIII
                            P[i][j]=find_mean(V)
                            print('Case3')
                            break
                        if only_0_and_MAX(v): #caseIV
                            P[i][j]=find_mean(v)
                            print('Case4')
                            break
                        if only_0(v): #caseV
                            P[i][j]=MAX
                            print('Case5')
                            break
                        if only_MAX(v): #caseVI
                            P[i][j]=0
                            print('Case6')
                            break
            # print(f"New P_i_j{P[i,j]}\n")
    return P

def denoise_image(image, filters):
    '''
    filters=[
        {name:filter1
            n_iter:...,
            params:{
                ...
            }
        },
        {name:filter2
            n_iter:...,
            params:{
                ...
            }
        },
        ...
    ]
    '''
    for filter in filters:
        for _ in range(filter['n_iter']):
            match filter['name']:
                case "mean":
                    '''
                    {"ksize" : 15}
                    '''
                    k_size = filter['params']['ksize']
                    mean_kernel = np.ones([k_size, k_size])/(k_size**2)
                    image = cv2.filter2D(image, -1, mean_kernel)

                case "median":
                    '''
                    {"ksize" : 11}
                    '''
                    image = cv2.medianBlur(image, **filter['params'])

                case "bilateral":
                    '''
                    {"d" : 3,
                    "sigmaColor" : 2,
                    "sigmaSpace" : 1.5}
                    '''
                    image = cv2.bilateralFilter(
                        image, **filter['params'])

                case 'gaussian':
                    '''
                    {"sigmaX" : 2}
                    '''
                    sigmaX = filter['params']['sigmaX']
                    ksize = 2*int(np.ceil(3*sigmaX)) + 1
                    image = cv2.GaussianBlur(image, (ksize, ksize), sigmaX)

                case 'non_local_means':
                    '''
                    {'templateWindowSize':5,"searchWindowSize":21, "h":3, "hColor": 20}
                    '''
                    image = cv2.fastNlMeansDenoisingColored(
                        image, **filter['params'])

                case 'canny':
                    '''
                    {'threshold1':5,"threshold2":21}
                    '''
                    image = cv2.Canny(image, **filter['params'])

                case "adaptive":
                    channel1 = image[:, :, 0]
                    channel2 = image[:, :, 1]
                    channel3 = image[:, :, 2]
                    image[:, :, 0] = adaptive_approach(channel1)
                    image[:, :, 1] = adaptive_approach(channel2)
                    image[:, :, 2] = adaptive_approach(channel3)

                case "adaptive2":
                    image = adaptive_davide(image)
                    # channel1 = image[:, :, 0]
                    # channel2 = image[:, :, 1]
                    # channel3 = image[:, :, 2]
                    # image[:, :, 0] = adaptive_davide(channel1)
                    # image[:, :, 1] = adaptive_davide(channel2)
                    # image[:, :, 2] = adaptive_davide(channel3)

                case 'sharpen':
                    kernel = np.array([[0, -1, 0],
                                       [-1, 5, -1],
                                       [0, -1, 0]])
                    image = cv2.filter2D(image, -1, kernel=kernel)

                case 'emboss':
                    kernel = np.array([[-2, -1, 0],
                                       [-1, 1, 1],
                                       [0, 1, 2]])
                    image = cv2.filter2D(image, -1, kernel=kernel)
                case _: pass
    return image

In [ ]:
def normalize(image):
    return (image.astype(float)*255.0/225).astype('uint8')

def diagonal(points,point2=None):
    if point2:
        return np.sqrt((points[0]-point2[0])**2+(points[1]-point2[1])**2)
    return np.sqrt(np.sum(np.subtract(points[:1,:],points[1:,:])**2))

def get_geometry(corners):
    return {'position': tuple(np.round((np.sum(corners,axis=0)/4)[0],0).astype(int)),
        'width': f'{max(diagonal(corners[1:3,:]),diagonal(np.vstack((corners[0,:], corners[3,:])))):0.0f}px',
        'height': f'{max(diagonal(corners[0:2,:]),diagonal(corners[2:4,:])):0.0f}px'
    }

In [ ]:
def display_used(settings):
    aux=[filter for filter in settings if ( 'matching_settings' in filter.keys() or filter['n_iter']>0)]
    display(aux)


def inside(center,image_shape): # center is (x;y), shape is (rows,columns)
    # return True
    return (center[0]>=0 and center[0]<image_shape[1]) and (center[1]>=0 and center[1]<image_shape[0])


def check_boundings(point,rectangle, tollerance=None):
    try:
        pointX,pointY=point
        centerX,centerY=rectangle['position']
        sizeX=int(rectangle['width'][:-2])
        sizeY=int(rectangle['height'][:-2])
        return (centerX-sizeX/2<=pointX and pointX<=centerX+sizeX/2) and (centerY-sizeY/2<=pointY and pointY<=centerY+sizeY/2)
    except ValueError:
        print(point,rectangle)
        return False

def flag_same_products_SI(results,method='overlapping_center',method_param=50,show_concentric_texts=False):
    '''
    per ogni scena 
        per ogni coppia di prodotti
            se hanno lo "stesso" centro flaggo a False quello con meno matches
    '''
    for scene in results.keys():
        keys=list(results[scene].keys())
        for product1 in keys:
            # results[scene][product1]["flag"]=True
            for product2 in [k for k in keys if k!=product1]:
                match method:
                    case 'center_vicinity':
                        if diagonal(results[scene][product1]["geometry"]["position"],results[scene][product2]["geometry"]["position"])<method_param:
                            if show_concentric_texts:
                                print(f'scene {scene}: products {product1} and {product2} are found to be concentric')
                            if results[scene][product1]["match_count"]>results[scene][product2]["match_count"]:
                                results[scene][product2]["flag"]=False
                            else:
                                results[scene][product1]["flag"]=False
                    case 'overlapping_center':
                        point=results[scene][product1]["geometry"]["position"] # tuple (x,y)
                        if check_boundings(point,results[scene][product2]["geometry"]):
                            if show_concentric_texts:
                                print(f'scene {scene}: products {product1} and {product2} are found to be concentric')
                            if results[scene][product1]["match_count"]>results[scene][product2]["match_count"]:
                                results[scene][product2]["flag"]=False
                            else:
                                results[scene][product1]["flag"]=False
                            
                    case _:
                        pass


def print_single_instances(results: dict,min_match,check_pos):
    max_product_len = max(len(str(product)) for scene in results for product in results[scene].keys())
    max_match_len = max(len(str(results[scene][product]["match_count"])) for scene in results for product in results[scene])
    max_x_len = max(len(str(results[scene][product]["geometry"]["position"][0])) for scene in results for product in results[scene] if inside(results[scene][product]['geometry']['position'], scene_info_a[scene-1][2]))
    max_y_len = max(len(str(results[scene][product]["geometry"]["position"][1])) for scene in results for product in results[scene] if inside(results[scene][product]['geometry']['position'], scene_info_a[scene-1][2]))
    max_width_len = max(len(str(results[scene][product]["geometry"]["width"])) for scene in results for product in results[scene])
    max_height_len = max(len(str(results[scene][product]["geometry"]["height"])) for scene in results for product in results[scene])

    for scene in results.keys():
        print(f'Scene {scene}:')
        for product in results[scene].keys():
            # print(results[scene][product]['geometry']['position'])
            if (results[scene][product]['match_count'] >= min_match and 
                 #set to false in `flag_concentric_refs` if refs refers to the same product (they both think to be the same product) 
                results[scene][product]['flag'] and
                (
                    (not check_pos) or 
                    inside(results[scene][product]['geometry']['position'], scene_info_a[scene-1][2])
                )  
                ):
                print(
                    f"\t\tInstance {i},"
                    + f" matches: {product['match_count']:<{max_match_len+1}},"
                    + f" position: ({product['geometry']['position'][0]:<{max_x_len}},"
                    + f" {product['geometry']['position'][1]:<{max_y_len-1}}),"
                    + f" width: {product['geometry']['width']:<{max_width_len-1}},"
                    + f" height: {product['geometry']['height']:<{max_height_len-1}}"
                )

                # print(f"\tProduct {f'{product},':<{max_product_len+1}}"
                #     + f" matches: {f'{results[scene][product]['match_count']},':<{max_match_len+1}}"
                #     + f" {{position: ({str(results[scene][product]['geometry']['position'][0]):<{max_x_len}},"
                #     + f" {f'{str(results[scene][product]['geometry']['position'][1])}':<{max_y_len-1}}),"
                #     + f" width: {f'{results[scene][product]['geometry']['width']}':<{max_width_len-1}},"
                #     + f" height: {results[scene][product]['geometry']['height']:<{max_height_len-1}}}}"
                #     )

solutions:
scene1:1,2  
scene2:3,4,5  
scene3:6,7,8  
scene4:1,(7),8,9,10  
scene5:(2),11,12  

## TRACK B

In [ ]:
SHOW_FILTERED = True
# Working settings:
# [{'name': 'median', 'n_iter': 2, 'params': {'ksize': 3}},
#  {'name': 'median', 'n_iter': 2, 'params': {'ksize': 5}},
#  {'name': 'median', 'n_iter': 1, 'params': {'ksize': 7}},
#  {'matching_settings': {'min_match': 40,
#    'check_position': 'overlapping_center'}}]
settings_b = [
    {'name': 'adaptive',
        'n_iter': 0,
     },
    {'name': "median",
        "n_iter": 2,
        "params": {"ksize": 3}
     },
    {'name': "median",
        "n_iter": 2,
        "params": {"ksize": 5}
     },
    {'name': "median",
        "n_iter": 1,
        "params": {"ksize": 7}
     },
    {'name': "median",
        "n_iter": 0,
        "params": {"ksize": 3}
     },
    {'name': "median",
        "n_iter": 0,
        "params": {"ksize": 5}
     },
    {'name': 'emboss',
        'n_iter': 0,
     },
    {'name': "median",
        "n_iter": 0,
        "params": {"ksize": 3}
     },
    {'name': "median",
        "n_iter": 0,
        "params": {"ksize": 9}
     },
    {'name': "gaussian",
        "n_iter": 0,
        "params": {"sigmaX": 1.5}
     },
    {'name': "mean",
        "n_iter": 0,
        "params": {"ksize": 15}
     },
    {'name': "bilateral",
        "n_iter": 0,
        "params": {"d": 9,
                   "sigmaColor": 150,
                   "sigmaSpace": 150}
     },
    {'name': "non_local_means",
        "n_iter": 0,
        "params": {
            'templateWindowSize': 9,
            "searchWindowSize": 21,
            "h": 3,
            "hColor": 20}
     },
    {'name': "canny",
        "n_iter": 0,
        "params": {
            'threshold1': 100,
            "threshold2": 200}
     },
]




In [ ]:
def flag_same_products_MI(results,method='overlapping_center',method_param=50,show_concentric_texts=False):
    '''
    per ogni scena 
        per ogni coppia di istanza
            se hanno lo "stesso" centro flaggo a False quella con meno matches
    '''
    for scene in results.keys():
        keys=list(results[scene].keys())
        for product1 in keys:
            for product2 in [k for k in keys if k != product1]:
                for instance1 in range(len(results[scene][product1])):
                    for instance2 in range(len(results[scene][product2])):
                        match method:
                            case 'center_vicinity':
                                if diagonal(results[scene][product1][instance1]["geometry"]["position"],results[scene][product2][instance2]["geometry"]["position"])<method_param:
                                    if show_concentric_texts:
                                        print(f'scene {scene}: products {product1} and {product2} are found to be concentric')
                                    if results[scene][product1][instance1]["match_count"]>results[scene][product2][instance2]["match_count"]:
                                        results[scene][product2][instance2]["flag"]=False
                                    else:
                                        results[scene][product1][instance1]["flag"]=False
                            case 'overlapping_center':
                                point=results[scene][product1][instance1]["geometry"]["position"] # tuple (x,y)
                                if check_boundings(point,results[scene][product2][instance2]["geometry"]):
                                    if show_concentric_texts:
                                        print(f'scene {scene}: products {product1} and {product2} are found to be concentric')
                                    if results[scene][product1][instance1]["match_count"]>results[scene][product2][instance2]["match_count"]:
                                        results[scene][product2][instance2]["flag"]=False
                                    else:
                                        results[scene][product1][instance1]["flag"]=False
                                    
                            case _:
                                pass


'''
    results_b={
        scene_index:{
            product_1_index:[
                {
                    'match_count':n,
                    'geometry':geom_dict (see get_geometry()),
                    'flag':Bool
                },
                {
                    'match_count':n,
                    'geometry':geom_dict (see get_geometry()),
                    'flag':Bool
                },
                ...
            ],
            ...
        },
        ...
    }
'''
def print_multiple_instances(results: dict,min_match,check_pos):
    max_product_len = max(len(str(product)) for scene in results for product in results[scene].keys())
    max_match_len = max(len(str(product["match_count"])) for scene in results for products in results[scene] for product in results[scene][products])
    max_x_len = max(len(str(product["geometry"]["position"][0])) for scene in results for products in results[scene] for product in results[scene][products] if inside(product['geometry']['position'], scene_info_b[scene-1][2]))
    max_y_len = max(len(str(product["geometry"]["position"][1])) for scene in results for products in results[scene] for product in results[scene][products] if inside(product['geometry']['position'], scene_info_b[scene-1][2]))
    max_width_len = max(len(str(product["geometry"]["width"])) for scene in results for products in results[scene] for product in results[scene][products])
    max_height_len = max(len(str(product["geometry"]["height"])) for scene in results for products in results[scene] for product in results[scene][products])

    for scene in results.keys():
        print(f'Scene {scene}:')
        for products in results[scene].keys():
            printable_prods=[]
            for product in results[scene][products]:
                if (product['match_count'] >= min_match and 
                    product['flag'] and 
                    (
                        (not check_pos) or 
                        inside(product['geometry']['position'], scene_info_b[scene-1][2])
                    )):
                    printable_prods.append(product)
            num_inside=len(printable_prods)
            if num_inside>0:
                print(f"\tProduct {products:<{max_product_len}} - {num_inside} instance found:")
                for i,product in enumerate(printable_prods,1):
                    print(f"\t\tInstance {i},"
                        + f" matches: {f'{product['match_count']},':<{max_match_len+1}}"
                        + f" {{position: ({str(product['geometry']['position'][0]):<{max_x_len}}"
                        + f", {f'{str(product['geometry']['position'][1])}':<{max_y_len-1}}),"
                        + f" width: {f'{product['geometry']['width']}':<{max_width_len-1}},"
                        + f" height: {product['geometry']['height']:<{max_height_len-1}}}}"
                        )
dicti={1: {
        1: [
            {'match_count': 509, 'geometry': {'position': (412, 539), 'width': '1049px', 'height': '803px'}, 'flag': True},
            {'match_count': 45, 'geometry': {'position': (412, 539), 'width': '1049px', 'height': '803px'}, 'flag': True}
            ],
        2: [{'match_count': 426, 'geometry': {'position': (1248, 533), 'width': '1047px', 'height': '803px'}, 'flag': True}],
        3: [{'match_count': 26, 'geometry': {'position': (1169, 1000), 'width': '300px', 'height': '1690px'}, 'flag': False}],
        4: [{'match_count': 30, 'geometry': {'position': (920, 642), 'width': '3px', 'height': '2px'}, 'flag': False}],
        5: [{'match_count': 81, 'geometry': {'position': (1246, 562), 'width': '1004px', 'height': '712px'}, 'flag': False}],
        6: [{'match_count': 13, 'geometry': {'position': (1457, 111), 'width': '314px', 'height': '1770px'}, 'flag': True}],
        7: [{'match_count': 13, 'geometry': {'position': (553486, 37480), 'width': '26242px', 'height': '1159px'}, 'flag': True}],
        8: [{'match_count': 21, 'geometry': {'position': (92, 801), 'width': '21px', 'height': '1px'}, 'flag': True}],
        9: [],
        10:[ {'match_count': 15, 'geometry': {'position': (510, 44), 'width': '310px', 'height': '25px'}, 'flag': True}],
        11:[],
        12:[ {'match_count': 7, 'geometry': {'position': (2499, -3400), 'width': '17176px', 'height': '19063px'}, 'flag': False}],
        13:[ {'match_count': 31, 'geometry': {'position': (1072, 508), 'width': '0px', 'height': '0px'}, 'flag': False}],
        14:[ {'match_count': 73, 'geometry': {'position': (1245, 616), 'width': '1037px', 'height': '888px'}, 'flag': False}]
    }}
# print_multiple_instances(results=dicti,min_match=1,check_pos=True)

In [ ]:
def remove_used(points: list, angles, coordinates):
    # print(angles)
    if angles.size == 0:
        return points

    aux = points.copy()
    geom = get_geometry(angles)
    # print(geom)
    for point in aux:
        if check_boundings(coordinates[point.trainIdx].pt, geom):
            # print(coordinates[point.trainIdx].pt)
            aux.remove(point)
    return aux


def remove_used2(points: list, mask, coordinates):
    aux = points.copy()
    count=0
    for i in range(len(mask)):
        if mask[i]==1:
            aux.pop(i-count)
            count+=1
    return aux


def print_masked(mask, points, coordinates):
    for i in range(len(mask)):
        if mask[i]==1:
            print(coordinates[points[i].trainIdx].pt,points[i])

In [ ]:
def remove_used3(keypoints, descriptors: np.array, angles):
    # print(angles)
    if angles.size == 0:
        return keypoints, descriptors
    keypoints = list(keypoints)
    geom = get_geometry(angles)
    count = 0
    for i in range(len(keypoints)):
        if check_boundings(keypoints[i-count].pt, geom):
            keypoints.pop(i-count)
            descriptors = np.delete(descriptors, i-count, 0)
            count += 1
    return tuple(keypoints), descriptors

In [ ]:
filters=[
    {'name': "median",
        "n_iter": 1,
        "params": {"ksize": 3}
     },
    {'name': "median",
        "n_iter": 0,
        "params": {"ksize": 5}
     },
    {'name': "median",
        "n_iter": 1,
        "params": {"ksize": 7}
     }]
img_scene=cv2.imread(scenes_b[-1])
# img_scene=normalize(img_scene)
img_scene=denoise_image(img_scene,filters)[:,0:,:]
img_prod=cv2.imread(products_b[3])

sift = cv2.xfeatures2d.SIFT_create()
kp_scene = sift.detect(img_scene)
kp_prod = sift.detect(img_prod)
# img_visualization = cv2.drawKeypoints(img_scene,kp_scene, None, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

plt.figure(figsize=(20,20))
plt.imshow(img_scene)
plt.show()
img_visualization = cv2.drawKeypoints(img_prod,kp_prod, None, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
# plt.imshow(img_visualization)
# plt.show()

kp_scene, des_scene = sift.compute(img_scene, kp_scene)
kp_prod, des_prod = sift.compute(img_prod, kp_prod)
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 150)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des_prod,des_scene,k=2)

good = []
for m,n in matches:
    if m.distance < 0.8*n.distance:
        good.append(m)

MIN_NUM_MATCHES = 5
box=np.array([])
while len(good)>MIN_NUM_MATCHES:
    # print(len(good))
    # good=remove_used(good,box,kp_scene)
    try:
        kp_scene,des_scene=remove_used3(kp_scene,des_scene,box)
        print(des_scene.shape[0])
        if des_scene.shape[0]<1:
            break
        matches = flann.knnMatch(des_prod,des_scene,k=2)
        good = []
        for m,n in matches:
            if m.distance < 0.8*n.distance:
                good.append(m)
        # print([(m.trainIdx,m.queryIdx) for m in good])

        # print(des_scene.shape,len(kp_scene))
        src_pts = np.float32([ kp_prod[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp_scene[m.trainIdx].pt for m in good]).reshape(-1,1,2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        matchesMask = mask.ravel().tolist()
        # print(np.sum(matchesMask))
        if np.sum(matchesMask)<MIN_NUM_MATCHES:
            break
        # print(matchesMask)
        # print_masked(matchesMask,good,kp_scene)

        h,w,z = img_prod.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
        box=dst
        # print(get_geometry(dst))
        img_scene_p = cv2.polylines(img_scene,[np.int32(dst)],True,255,3, cv2.LINE_AA)
        draw_params = dict(matchColor = (255,0,0), # draw matches in red color
                        singlePointColor = None, # not draw keypoints only matching lines
                        matchesMask = matchesMask, # draw only inliers
                        flags = 2) # not draw keypoints only lines
        img3 = cv2.drawMatches(img_visualization,kp_prod,img_scene,kp_scene,good,None,**draw_params)
        plt.figure(figsize=(15,15))
        extent=[ -img_prod.shape[1], img_scene.shape[1],img_prod.shape[0],0]
        # print(extent)
        # plt.grid()
        # plt.xticks(np.arange(-410, 1696,75))
        plt.imshow(img3, extent=extent)
        plt.show()
        # good=remove_used2(good,matchesMask,kp_scene)
    except cv2.error:
        # print(des_scene.shape)
        print('cv2 error',cv2.error.msg)
    except IndexError:
        pass

In [ ]:

plt.figure(figsize=(20,20))
img_scene=cv2.imread(scenes_b[-1], cv2.IMREAD_GRAYSCALE)
plt.imshow(img_scene, cmap='gray')

In [ ]:
filters=[
    {'name': "median",
        "n_iter": 1,
        "params": {"ksize": 3}
     },
    {'name': "median",
        "n_iter": 0,
        "params": {"ksize": 5}
     },
    {'name': "median",
        "n_iter": 1,
        "params": {"ksize": 7}
     },
    {'name': "adaptive2",
        "n_iter": 0
     }]


plt.figure(figsize=(20,20))
img_scene=cv2.imread(scenes_b[-1])
img_scene=denoise_image(img_scene,filters)
plt.subplot(221)
plt.imshow(img_scene[:,:,0], cmap='gray')
plt.subplot(222)
plt.imshow(img_scene[:,:,1], cmap='gray')
plt.subplot(223)
plt.imshow(img_scene[:,:,2], cmap='gray')
plt.show()

In [ ]:
filters=[
    {'name': "median",
        "n_iter": 1,
        "params": {"ksize": 3}
     },
    {'name': "median",
        "n_iter": 0,
        "params": {"ksize": 5}
     },
    {'name': "median",
        "n_iter": 1,
        "params": {"ksize": 7}
     }]
plt.figure(figsize=(20,20))
img_scene = cv2.imread(scenes_b[-1])
img_scene=denoise_image(img_scene,filters)  
kp_scene = sift.detect(img_scene)
kp_scene, des_scene = sift.compute(img_scene, kp_scene)
img_kp = cv2.drawKeypoints(img_scene, kp_scene, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.imshow(cv2.cvtColor(img_kp, cv2.COLOR_BGR2RGB))

In [ ]:
def remove_used4(keypoints,descriptors,angles):
    if angles.size == 0:
        return keypoints, descriptors
    removed=0
    keypoints = list(keypoints)
    angles=[tuple(row[0]) for row in angles]
    for ikp in range(len(keypoints)):
        px, py = keypoints[ikp-removed].pt
        count = 0
        n = 4
        remove=False
        for i in range(n):
            x1, y1 = angles[i]
            x2, y2 = angles[(i + 1) % n]

            # Check if the point is exactly on a vertex
            if (px, py) == (x1, y1) or (px, py) == (x2, y2):
                remove= True

            # Check if the point is on the edge
            if y1 != y2 and min(y1, y2) <= py <= max(y1, y2):
                x_intercept = (py - y1) * (x2 - x1) / (y2 - y1) + x1
                if px == x_intercept:
                    remove= True

            # Check intersections with a ray from the point
            if (y1 <= py < y2) or (y2 <= py < y1):
                x_intercept = (py - y1) * (x2 - x1) / (y2 - y1) + x1
                if px < x_intercept:
                    count += 1

        if count % 2 == 1 or remove:
            # print(f'point:{keypoints[i-removed].pt}, angles:{angles}')
            keypoints.pop(ikp-removed)
            descriptors = np.delete(descriptors, ikp-removed, 0)
            removed += 1
    return tuple(keypoints), descriptors

In [ ]:
filters=[
    {'name': "median",
        "n_iter": 2,
        "params": {"ksize": 3}
     },
    {'name': "median",
        "n_iter": 0,
        "params": {"ksize": 5}
     },
    {'name': "median",
        "n_iter": 1,
        "params": {"ksize": 7}
     }]

prod=[products_b[i] for i in [3,-1,-2]]

#plt.figure(figsize=(20,20))
img_scene=cv2.imread(scenes_b[-1])
img_scene=denoise_image(img_scene,filters)
#plt.imshow(cv2.cvtColor(img_scene,cv2.COLOR_BGR2RGB))
#plt.show()
for product in prod:
    print(f'product {product}')
    img_prod = cv2.imread(product)
    sift = cv2.xfeatures2d.SIFT_create()
    
    for channel in range(3):
        print(f'channel {channel +1}')
        img_scene = cv2.imread(scenes_b[-1])
        img_scene=denoise_image(img_scene,filters)
        
        kp_scene = sift.detect(img_scene[:,:,channel])
        kp_prod = sift.detect(img_prod[:,:,channel])
        kp_scene, des_scene = sift.compute(img_scene[:,:,channel], kp_scene)
        kp_prod, des_prod = sift.compute(img_prod[:,:,channel], kp_prod)
        
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks = 150)
        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des_prod,des_scene,k=2)

        good = []
        for m,n in matches:
            if m.distance < 0.8*n.distance:
                good.append(m)

        MIN_NUM_MATCHES = 5
        box=np.array([])
        while len(good)>MIN_NUM_MATCHES:
            try:
                kp_scene,des_scene=remove_used3(kp_scene,des_scene,box)
                # plt.figure(figsize=(20,20))
                img_kp = cv2.drawKeypoints(img_scene[:,:, channel], kp_scene, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
                plt.imshow(cv2.cvtColor(img_kp, cv2.COLOR_BGR2RGB))
                plt.show()
                if des_scene.shape[0]<1:
                    break
                matches = flann.knnMatch(des_prod,des_scene,k=2)
                good = []
                for m,n in matches:
                    if m.distance < 0.8*n.distance:
                        good.append(m)

                src_pts = np.float32([ kp_prod[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
                dst_pts = np.float32([ kp_scene[m.trainIdx].pt for m in good]).reshape(-1,1,2)

                M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

                matchesMask = mask.ravel().tolist()
                print(np.sum(matchesMask),des_scene.shape[0])
                if np.sum(matchesMask)<MIN_NUM_MATCHES:
                    break

                h,w = img_prod[:,:,channel].shape
                pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
                dst = cv2.perspectiveTransform(pts,M)
                box=dst
                
                img_scene_p = cv2.polylines(img_scene,[np.int32(dst)],True,0,3, cv2.LINE_AA)
                draw_params = dict(matchColor = (0,0,0), # draw matches in red color
                                singlePointColor = None, # not draw keypoints only matching lines
                                matchesMask = matchesMask, # draw only inliers
                                flags = 2) # not draw keypoints only lines
                #img3 = cv2.drawMatches(img_prod[:,:,channel],kp_prod,img_scene[:,:,channel],kp_scene,good,None,**draw_params)
                # extent=[ -img_prod.shape[1], img_scene.shape[1],img_prod.shape[0],0]
                # #plt.imshow(img3, 'gray',extent=extent)
                # plt.show()
                
            except cv2.error:
                print('cv2 error',cv2.error.msg)
            except IndexError:
                pass
    plt.show()